# Hackapizza 2025


## Imports


In [ ]:
import ast
import csv
import re
from pathlib import Path

import nest_asyncio
import pandas as pd
from dotenv import load_dotenv
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from llama_index.core import SimpleDirectoryReader
from llama_parse import LlamaParse


In [ ]:
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key="AnzfgthfcrfRzttoXGiKZUJDMRlcB3w4uemf0PJGFFT5"
)

OVERWRITE = False


In [ ]:
model = ModelInference(
    model_id="mistralai/mistral-large",  # Che conosciamo bene 😊🏆
    credentials=credentials,
    project_id="5c33debb-5a25-4bfe-8392-ede4b20884fe",
    params={
        "max_tokens": 200
    }
)


In [ ]:
# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

# bring in our LLAMA_CLOUD_API_KEY
load_dotenv()

# set up parser
parser = LlamaParse(
    result_type="text"  # "markdown" and "text" are available
)

# Define the folder path
folder_path = Path('./data/Menu')

menus = {}

# Iterate through all files in the folder
for filename in folder_path.iterdir():
    if filename.is_file():
        # use SimpleDirectoryReader to parse our file
        file_extractor = {".pdf": parser}
        documents = SimpleDirectoryReader(input_files=[filename], file_extractor=file_extractor).load_data()
        doc_text = ""

        for i in range(0, len(documents)):
            doc_text += documents[i].text

        menus[filename] = doc_text


In [ ]:
df = pd.DataFrame(columns=["FileName", "Text"])

for key, text in menus.items():
    key_name = key.name.split('/')[-1]  # Split the string by '/' and take the last part
    new_row = pd.DataFrame([[key_name, text]], columns=["FileName", "Text"])
    df = pd.concat([df, new_row], ignore_index=True)


In [ ]:
print(df)


In [ ]:
with open("./output/menus.csv", 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    for key, value in menus.items():
        key_name = key.name.split('/')[-1]  # Split the string by '/' and take the last part
        writer.writerow([key_name, value])


In [ ]:
MENUS = Path('output/menus.csv')
menus = pd.read_csv(filepath_or_buffer=MENUS, header=None, names=['FileName', 'Text'])
menus_metadata_df = pd.DataFrame(columns=['Ristorante', 'Pianeta', 'Chef', 'Licenze'])

for idx, menu in menus.iterrows():
    content = menu['Text'].split(sep='Menu', maxsplit=1)[0]
    result = model.chat(messages=[
        {
            'role': 'system',
            'content': "Estrai i metadata dal seguente testo individuando il nome del ristorante, il nome del pianeta, il nome dello chef, e le licenze del ristorante e scrivi nel seguente modo la tua risposta rimpiazzando solo i valori dei campi, come un dizionario:\n{\"Ristorante\": nome_ristorante, \"Pianeta\": nome_pianeta, \"Chef\": nome_chef, \"Licenze\": lista_di_licenze}. Se non trovi valore in uno dei campi, scrivi la stringa vuota \"\"."
        },
        {
            'role': 'user',
            'content': content
        }
    ])
    ans = re.sub(pattern=r'^[^\{]*', repl='', string=result['choices'][0]['message']['content'])
    ans = re.sub(pattern=r'[^\}]*$', repl='', string=ans)

    try:
        menus_metadata_df = pd.concat([menus_metadata_df, pd.DataFrame([ast.literal_eval(ans)])])
    except:
        print(ans)

        break

if OVERWRITE:
    menus_metadata_df.to_pickle(path=Path('data/menus_metadata.pkl'))
